In [ ]:
# !sudo /opt/conda3/bin/pip install -i http://192.168.101.40/pypi/simple --trusted-host 192.168.101.40 -r utils/pip_list.txt
from main.utils.analysis import *
from main.utils.hyperopt_lgb import *
from main.utils.global_parameters import *
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

class auto_ML(object):
    
    def __init__(self):
        try:
            Path(params4eda.get('store_pth')+"/eda results").mkdir(parents=True, exist_ok=True)
            print(f"eda results文件夹已就绪")
        except Exception as e:
            print(f'{e} eda results文件夹生成失败')
            return
        
        print("step 1 获取数据")
        self.data = get_dataset(**{'data_pth': params4eda.get('data_pth'), 'date_colName': params4eda.get('date_colName'),
                                 'y_colName': params4eda.get('y_colName'), 'org_colName': params4eda.get('org_colName'),
                                 'data_encode': params4eda.get('data_encode'), 'key_colNames': params4eda.get('key_colNames')})
        self.data_analysis = org_analysis(self.data)
        self.data_analysis.to_csv(params4eda.get('store_pth')+"/eda results/"+"全量样本分机构信息概览.csv", index=False)
        
        self.oos_data = self.data[self.data.new_org.isin(params4eda.get('oos_orgs'))]
        self.data = self.data[~self.data.new_org.isin(params4eda.get('oos_orgs'))]
        self.data = drop_abnormal_ym(**{'data':self.data, 'minYmBadsample': params4eda.get('minYmBadsample'), 
                                 'minYmSample':params4eda.get('minYmSample')})
        print(f'建模样本[去除异常月份]大小为{self.data.shape[0]}, 贷外样本[不做处理]大小为{self.oos_data.shape[0]}')

        print("step 2 去除高缺失率变量")
        self.miss_org, self.miss_channel = missing_check(**{'data': self.data, 'channel': params4eda.get('channels')})
        self.miss_org.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量分机构缺失率.csv", index=False)
        self.miss_channel.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量分渠道缺失率.csv", index=False)
        self.data = drop_highmiss_features(**{'data':self.data, 'ratio': params4eda.get('missing_ratio'), 
                                              'cnt': params4eda.get('missing_orgcnt'), 'miss_org': self.miss_org,
                                              'miss_channel': self.miss_channel})
        
        print("step 3 去除低iv变量")
        self.res_iv_org, self.res_iv_channel = detect_iv(**{'data':self.data, 'method':params4eda.get('bin_method'), 
                                                            'bins':params4eda.get('bin_cnt'),
                                                            'channel': params4eda.get('channels')})
        self.res_iv_org.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量分机构iv.csv", index=False)
        self.res_iv_channel.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量分渠道iv.csv", index=False)
        self.data = drop_lowiv_features(**{'data':self.data, 'miniv_org':params4eda.get('miniv_org'), 
                               'miniv_channel':params4eda.get('miniv_channel'), 'cnt':params4eda.get('lowiv_orgcnt'),
                               'res_iv_org':self.res_iv_org, 'res_iv_channel': self.res_iv_channel})
        
        print("step 4 去除高psi变量")
        self.res_psi_org, self.res_psi_channel = detect_psi(**{'data': self.data, 'channel': params4eda.get('channels')})
        self.res_psi_org.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量分机构psi.csv", index=False)
        self.res_psi_channel.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量分渠道psi.csv", index=False)
        self.data = drop_highpsi_features(**{'data':self.data, 'res_psi_org':self.res_psi_org,
                                             'cnt':params4eda.get('highpsi_orgcntnt'), 'ratio':params4eda.get('max_psi')})
        
        print("step 5 去除高相似性变量")
        self.indices, self.corr = detect_correlation(**{'data':self.data, 'method':params4eda.get('corr_method'),
                                                        'max_corr':params4eda.get('max_corr')})
        self.indices = pd.DataFrame(self.indices)
        self.indices.to_csv(params4eda.get('store_pth')+"/eda results/"+"高相似变量对.csv", index=False)
        self.corr.to_csv(params4eda.get('store_pth')+"/eda results/"+"变量相似性矩阵.csv", index=False)
        self.indices = pd.read_csv(params4eda.get('store_pth')+"/eda results/"+"高相似变量对.csv")
        self.data = drop_highcorrelation_features(**{'data':self.data,
                                                     'res_iv_channel':self.res_iv_channel,'indices':self.indices, 
                                                     'channel':'整体'})
        
        print("step 6 null importance去除高噪音变量")
        self.data = drop_highnoise_features(**{'data': self.data, 'n_estimators':100, 'max_depth': 5})[0]
        pd.DataFrame(self.data.columns, index=['features']).to_csv(params4eda.get('store_pth')+"/eda results/"+"筛选得到的变量.csv")
        print("EDA 分析&处理完毕")
        
    def finetuingParams(self):
        try:
            Path(params4eda.get('store_pth')+"/hyperopt results").mkdir(parents=True, exist_ok=True)
            print(f"hyperopt results文件夹已就绪")
        except Exception as e:
            print(f'{e} hyperopt results文件夹生成失败')
            return
        
        print("建模数据data执行异常值替换操作, -999 -1111 -1被替换为NaN")
        self.data.replace({-1111:np.nan, -999:np.nan, -1:np.nan}, inplace=True)
        params4hyperopt['data'] = self.data
        params4hyperopt['record_train_process'] = False
        optlgb = HyperOptLGB(**params4hyperopt)
        _ = optlgb.tpesearch_params()
        
        best_loss = 0
        for idx in np.arange(len(optlgb.trails.results)):
            if optlgb.trails.results[idx].get('loss') < best_loss:
                best_loss = optlgb.trails.results[idx].get('loss')
        for idx in np.arange(len(optlgb.trails.results)):
            if optlgb.trails.results[idx].get('loss') == best_loss:
                self.param = optlgb.trails.results[idx].get('param')
        pd.DataFrame(self.param, index=['0']).to_csv(params4eda.get('store_pth')+"/hyperopt results/"+"最优参数.csv")
        print(f"最优参数为: {self.param}, oos mean ks为{best_loss}")
        
        self.param_ = {
            key: hp.choice(key, [value]) for key, value in self.param.items()
        }
        params4hyperopt.update({'params': self.param_, 'max_iterations': 1, 'record_train_process':True})
        
        ## 重新拟合获得训练日志
        print("重新拟合获得训练日志中")
        optlgb1= HyperOptLGB(**params4hyperopt)
        _ = optlgb1.tpesearch_params()
        
        self.simpler_results = optlgb1.trails.results[0].get('simpler_results')
        self.results = optlgb1.trails.results[0].get('results')
        self.results['idx'] = self.results.index
        
        self.simpler_results.to_csv(params4eda.get('store_pth')+"/hyperopt results/"+"simpler_results.csv", index=False)
        self.results.to_csv(params4eda.get('store_pth')+"/hyperopt results/"+"results.csv", index=False)
        print("参数寻优完成")
        
    def inference(self):
        ## 重新读取results, 保存后的results发生了索引变化
        results = pd.read_csv(params4eda.get('store_pth')+"/hyperopt results/"+"results.csv")
        params4inference.update({'data':self.data, 'oos_data':self.oos_data, 
                                 'param': self.param, 'randn': params4hyperopt.get('randn'),
                                 'results': results, 'dataset_statis': self.data_analysis,
                                 'store_pth':params4eda.get('store_pth')})
        inference = Inference(**params4inference)
        inference.generate_report()

In [ ]:
auto_ml = auto_ML()

In [ ]:
auto_ml.data.head(1)

In [ ]:
auto_ml.finetuingParams()

In [ ]:
auto_ml.inference()